In [ ]:
from models import api_models
import numpy as np
import pandas as pd
import pickle
import gc
from sklearn.neural_network import MLPClassifier
from tensorflow import keras
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from captain_mode_draft import Draft

# prepare data

In [ ]:
data = api_models.load_from_msgpack("input/scrap_merge.msgpack.gz")
data_x = []
data_y = []
for data_item in data:
    input_vector = np.zeros(130)
    for player in data_item.players:
        if player.team == 1:
            input_vector[player.hero_id]=1
        else:
            input_vector[player.hero_id]=-1
    if data_item.radiant_win == True:
          y = 1
    else:
          y=-1
    data_x.append(input_vector)
    data_y.append(y)

arr = []
for y in data_y:
    if y == 1:
        arr.append([1, 0])
    else:
        arr.append([0, 1])

del data
data_y = arr
pickle.dump( data_y, open( "input/data_y.pkl", "wb" ) )
pickle.dump( data_x, open( "input/data_x.pkl", "wb" ) )

# load data

In [ ]:
#data_y = pickle.load( open( "input/data_y.pkl", "rb" ) )
#data_x = pickle.load( open( "input/data_x.pkl", "rb" ) )
#x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, random_state=1)

# model train

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(130, input_shape=(130,), activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size = 64, verbose=1)

y_pred = model.predict(np.array(x_test))
tmp_y_pred = []
for y in y_pred:
    if y[0] > y[1]:
        tmp_y_pred.append([1, 0])
    else:
        tmp_y_pred.append([0, 1])
accuracy_score(y_test, tmp_y_pred)

In [ ]:
model.save("models_win_rate/130_50_relu")

# simulation

In [1]:
import sys

import random
import numpy as np
import time
from utils.parser import parse_mcts_exp_parameters
from captain_mode_draft import Draft
from tensorflow import keras
import logging
from models_win_rate.model_win_rate import ModelWinRateKeras130

def experiment(match_id, p0_model_str, p1_model_str, model_win_rate):
    t1 = time.time()
    d = Draft(model_win_rate = model_win_rate, p0_model_str=p0_model_str, p1_model_str=p1_model_str)  # instantiate board

    while not d.end():
        p = d.get_player()
        t2 = time.time()
        
        mt = d.decide_move_type() # whether it is ban or pick
        a = p.get_best_move(mt)
        d.move(a)
        d.print_move(match_id=match_id, move_duration=time.time() - t2, move_id=a, move_type=mt)
    
    final_red_team_win_rate = d.eval()
    duration = time.time() - t1
    exp_str = 'match: {}, time: {:.3F}, red team predicted win rate: {:.5f}' \
        .format(match_id, duration, final_red_team_win_rate)
    
    return final_red_team_win_rate, duration, exp_str

In [2]:
logger = logging.getLogger('mcts')
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.WARNING)
num_matches = 1
p0_model_str = 'random'      
p1_model_str = 'mcts_50_0.5'            # format: mcts_maxiterations_c

model_win_rate = ModelWinRateKeras130() # win rate model. input size (130)
model_win_rate.load('models_win_rate/130_50_relu')

red_team_win_rates, times = [], []
for i in range(num_matches):
    wr, t, s = experiment(i, p0_model_str, p1_model_str, model_win_rate)
    red_team_win_rates.append(wr)
    times.append(t)
    s += ', mean predicted win rate: {:.5f}\n'.format(np.average(red_team_win_rates))
    logger.warning(s)

logger.warning('{} matches, {} vs. {}. average time {:.5f}, average red team win rate {:.5f}, std {:.5f}'
               .format(num_matches, p0_model_str, p1_model_str,
                       np.average(times), np.average(red_team_win_rates), np.std(red_team_win_rates)))


match 0 player 0 (random         ), ban :  44, move_cnt: 1, duration: 0.000
match 0 player 1 (mcts_50_0.5    ), ban :  82, move_cnt: 1, duration: 2.523
match 0 player 0 (random         ), ban :   8, move_cnt: 2, duration: 0.000
match 0 player 1 (mcts_50_0.5    ), ban :  42, move_cnt: 2, duration: 2.392
match 0 player 0 (random         ), ban :  47, move_cnt: 3, duration: 0.000
match 0 player 1 (mcts_50_0.5    ), ban :  81, move_cnt: 3, duration: 2.463
match 0 player 0 (random         ), pick:  70, move_cnt: 4, duration: 0.000
match 0 player 1 (mcts_50_0.5    ), pick:  97, move_cnt: 4, duration: 2.371
match 0 player 1 (mcts_50_0.5    ), pick: 121, move_cnt: 5, duration: 2.452
match 0 player 0 (random         ), pick:  24, move_cnt: 5, duration: 0.000
match 0 player 1 (mcts_50_0.5    ), ban : 107, move_cnt: 6, duration: 2.617
match 0 player 0 (random         ), ban :  59, move_cnt: 6, duration: 0.000
match 0 player 1 (mcts_50_0.5    ), ban : 116, move_cnt: 7, duration: 2.434
match 0 play

# demo recommendation
    

In [4]:
from pick_recomendation import PicksRecommendation
from models_win_rate.model_win_rate import ModelWinRateKeras130

model_win_rate = ModelWinRateKeras130()
model_win_rate.load('models_win_rate/130_50_relu')
p0_model_str = 'mcts_10_0.5'   # rekomendacja dla "gracza"(druzyny) 1, maxiter in mcts= 10, c = 0.5
p1_model_str = 'mcts_10_0.5'  # rekomendacja dla "gracza" 2
recomm = PicksRecommendation(model_win_rate=model_win_rate, 
                             p0_model_str=p0_model_str,
                             p1_model_str=p1_model_str)


In [ ]:
state_picks = [[],[]]
state_bans  = [[],[]]
for i in range(25):
    print("Stan planszy: ")
    print("G1 pick: ", state_picks[0]) #+[-1]*(5-len(state_picks[0])))
    print("G1 bans: ", state_bans[0])  #+[-1]*(6-len(state_bans[0])))
    print("G2 pick: ", state_picks[1]) #+[-1]*(5-len(state_picks[1])))
    print("G2 bans: ", state_bans[1])  #+[-1]*(6-len(state_bans[1])))
    
    type_move, id_player, moves = recomm.recommend_pick(state_picks, state_bans, k=5)
    #print(type_move, id_player, moves)
    print("Nastepny ruch G",id_player+1, " ", type_move)
    for x in moves:
        print(x[0], x[1])
    x=int(input("Wpisz id bohatera: "))
    
    if type_move == 'ban':
        state_bans[id_player].append(x)
    else:
        state_picks[id_player].append(x)
    print("")
    #print(state_picks, state_bans)
    

Stan planszy: 
G1 pick:  []
G1 bans:  []
G2 pick:  []
G2 bans:  []
Nastepny ruch G 1   ban
5 0.5120272040367126
87 0.48463568091392517
125 0.48238977789878845
119 0.3390335738658905
55 0.3183991312980652
Wpisz id bohatera: 5

Stan planszy: 
G1 pick:  []
G1 bans:  [5]
G2 pick:  []
G2 bans:  []
Nastepny ruch G 2   ban
62 0.9364078640937805
125 0.7670018970966339
114 0.7343661785125732
23 0.5555130541324615
60 0.550444483757019
Wpisz id bohatera: 11

Stan planszy: 
G1 pick:  []
G1 bans:  [5]
G2 pick:  []
G2 bans:  [11]
Nastepny ruch G 1   ban
55 0.816818356513977
86 0.6636686325073242
32 0.5899741053581238
103 0.5769644379615784
39 0.5483741760253906
Wpisz id bohatera: 55

Stan planszy: 
G1 pick:  []
G1 bans:  [5, 55]
G2 pick:  []
G2 bans:  [11]
Nastepny ruch G 2   ban
76 0.7839206606149673
87 0.6521320641040802
121 0.5629115998744965
35 0.562593400478363
49 0.5443386435508728
Wpisz id bohatera: 76

Stan planszy: 
G1 pick:  []
G1 bans:  [5, 55]
G2 pick:  []
G2 bans:  [11, 76]
Nastepny ruc